# Part 2: Intro to Federated Learning

In the last section, we learned about PointerTensors, which create the underlying infrastructure we need for privacy preserving Deep Learning. In this section, we're going to see how to use these basic tools to implement our first privacy preserving deep learning algorithm, Federated Learning.

### What is Federated Learning?

It's a simple, powerful way to train Deep Learning models. If you think about training data, it's always the result of some sort of collection process. People (via devices) generate data by recording events in the real world. Normally, this data is aggregated to a single, central location so that you can train a machine learning model. Federated Learning turns this on its head!

Instead of bringing training data to the model (a central server), you bring the model to the training data (wherever it may live).

The idea is that this allows whoever is creating the data to own the only permanent copy, and thus maintain control over who ever has access to it. Pretty cool, eh?

# Section 2.1 - A Toy Federated Learning Example

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

Note: If this API is un-familiar to you - head on over to [fast.ai](http://fast.ai) and take their course before continuing in this tutorial.

In [4]:
import syft as sy
hook = sy.TorchHook()
from torch import nn, optim

# A Toy Dataset
data = sy.Var(sy.FloatTensor([[0,0],[0,1],[1,0],[1,1]]))
target = sy.Var(sy.FloatTensor([[0],[0],[1],[1]]))

# A Toy Model
model = nn.Linear(2,1)

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much the missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data[0])

In [5]:
train()

0.7935236096382141
0.21217259764671326
0.12388776242733002
0.08632121235132217
0.06183892861008644
0.044710226356983185
0.0325467512011528
0.02383730188012123
0.017555570229887962
0.012994087301194668
0.009660916402935982
0.007211238145828247
0.005401470698416233
0.004058181773871183
0.003056995337828994
0.002308045281097293
0.0017459806986153126
0.0013229941250756383
0.0010039110202342272
0.0007627044105902314


And there you have it! We've trained a basic model in the conventional manner. All our data is aggregated into our local machine and we can use it to make updates to our model. Federated Learning, however, doesn't work this way. So, let's modify this example to do it the Federated Learning way! 

So, what do we need:

- create a couple workers
- get pointers to training data on each worker
- updated training logic to do federated learning

    New Training Steps:
    - send model to correct worker
    - train on the data located there
    - get the model back and repeat with next worker

In [10]:
# create a couple workers

bob = sy.VirtualWorker(id="bob")
alice = sy.VirtualWorker(id="alice")

# get pointers to training data on each worker by
# sending some training data to bob and alice
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

data_alice = data[2:].send(alice)
target_alice = target[2:].send(alice)

# organize pointers into a list
datasets = [(data_bob,target_bob),(data_alice,target_alice)]

# Iniitalize A Toy Model
model = nn.Linear(2,1)

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):
        
        # NEW) iterate through each worker's dataset
        for data,target in datasets:
            
            # NEW) send model to correct worker
            model.send(data.location)

            # 1) erase previous gradients (if they exist)
            opt.zero_grad()

            # 2) make a prediction
            pred = model(data)

            # 3) calculate how much the missed
            loss = ((pred - target)**2).sum()

            # 4) figure out which weights caused us to miss
            loss.backward()

            # NEW) get model (with gradients)
            model.get()

            # 5) change those weights
            opt.step()

            # 6) print our progress
            print(loss.get().data[0]) # NEW) slight edit... need to call .get() on loss

In [11]:
train()

0.49059900641441345
0.48288848996162415
0.5257140398025513
0.20319302380084991
0.3727419972419739
0.14014576375484467
0.2604595422744751
0.10102146863937378
0.18343526124954224
0.07364650815725327
0.13027068972587585
0.05409587174654007
0.0932079330086708
0.039966512471437454
0.06712788343429565
0.029659217223525047
0.04862105846405029
0.022084159776568413
0.03538985550403595
0.016484789550304413
0.02586808241903782
0.012327495031058788
0.018976470455527306
0.00923063326627016
0.013963752426207066
0.006918045692145824
0.010302114300429821
0.005188031122088432
0.007617602590471506
0.0038922084495425224
0.005643305368721485
0.002920745871961117
0.004187457729130983
0.0021920367144048214
0.0031114660669118166
0.0016452015843242407
0.0023146746680140495
0.0012347802985459566
0.001723655266687274
0.0009266938432119787


## Well Done!

And voila! We now are training a very simple Deep Learning model using Federated Learning! We send the model to each worker, generate a new gradient, and then bring the gradient back to our local server where we update our global model. Never in this process do we ever see or request access to the underlying training data! We preserve the privacy of Bob and Alice!!!

## Shortcomoings of this Example

So, while this example is a nice introduction to Federated Learning, it still has some major shortcomings. Most notably, when we call model.get() and receive the updated model from Bob or Alice, we can actually learn a lot about Bob and Alice's training data by looking at their gradiets. In some cases, we can restore their training data perfectly! 

So, what is there to do? Well, the first strategy people employ is to **average the gradient across multiple individuals before uploading it to the central server**. This strategy, however, will require some more sophisticated use of PointerTenor objects. So, in the next section, we're going to take some time to learn about more advanced pointer functionality and then we'll upgrade this Federated Learning example
